Reference: https://github.com/xlang-ai/instructor-embedding/tree/main/examples/faiss

In [ ]:
!pip install InstructorEmbedding

In [ ]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 12.0 MB/s eta 0:00:00


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=09e3b4ed04d094502839f9a59d1f718e9a387e4e78a58307a814b7b291528a20
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
from InstructorEmbedding import INSTRUCTOR
model_ins = INSTRUCTOR('hkunlp/instructor-base')

sentence = "3D ActionSLAM: wearable person tracking in multi-floor environments"
instruction = "Represent the Science title:"
embeddings = model_ins.encode([[instruction,sentence]])
print(embeddings.shape)

load INSTRUCTOR_Transformer
max_seq_length  512
(1, 768)


In [ ]:

import json
import numpy as np
import faiss
import torch

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ", device)

device =  cuda


In [16]:
!wget https://raw.githubusercontent.com/xlang-ai/instructor-embedding/main/examples/faiss/arxiv-metadata-10000.json

--2023-09-23 04:59:29--  https://raw.githubusercontent.com/xlang-ai/instructor-embedding/main/examples/faiss/arxiv-metadata-10000.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14576266 (14M) [text/plain]
Saving to: ‘arxiv-metadata-10000.json.1’

arxiv-metadata-1000 100%[===================>]  13.90M  --.-KB/s    in 0.1s    

2023-09-23 04:59:30 (133 MB/s) - ‘arxiv-metadata-10000.json.1’ saved [14576266/14576266]



In [17]:

# FAISS index setup
dimension = 768  # Instructor-XL output dimension
index_ins = faiss.IndexFlatL2(dimension)

# Extract and vectorize data
db_filename = 'arxiv-metadata-10000.json.1'
num_lines = 10000
batch_size = 4


In [18]:
# Load all papers from JSON
with open(db_filename, 'r') as f:
    papers = [json.loads(line) for line in f]

# Extract the papers' titles and abstracts
texts = [f"{paper['title']}: {paper['abstract']}" for paper in papers]

# Preparation for encoding
instructions = ["Represent the science titles and abstracts: "] * len(texts)

# Prepare the inputs
inputs = [[instr, txt] for instr, txt in zip(instructions, texts)]

In [19]:
# Create vectors using Instructor
vectors = model_ins.encode(
    sentences=inputs[:num_lines],
    batch_size=batch_size,
    show_progress_bar=True,
    convert_to_numpy=True,
    device=str(device)
)


Batches:   0%|          | 0/2500 [00:00<?, ?it/s]

In [20]:
# Add the vectors to the FAISS index
index_ins.add(np.array(vectors).astype('float32'))

print(f"Added {num_lines} papers to the FAISS index.")


Added 10000 papers to the FAISS index.


In [21]:
# --------------------------------------------------------------------------------

def search_ins(query, k=5):
    vector = model_ins.encode(["Represent the query to a science database: ", query])
    _, indices = index_ins.search(np.array(vector[1]).reshape(1, -1).astype('float32'), k)
    return indices[0]



In [22]:
queries = ['Machine learning model for forex trading']

In [32]:
def print_paper_details(line_numbers):
  for line_number in line_numbers:
    print(papers[line_number]['title'])
    print(papers[line_number]['abstract'])

In [33]:
for query in queries:
    print(f"Question: {query}\n")
    line_numbers = search_ins(query, k=2)
    print_paper_details(line_numbers)
    print('-'*80)

Question: Machine learning model for forex trading

Inferring the Composition of a Trader Population in a Financial Market
  We discuss a method for predicting financial movements and finding pockets of
predictability in the price-series, which is built around inferring the
heterogeneity of trading strategies in a multi-agent trader population. This
work explores extensions to our previous framework (arXiv:physics/0506134).
Here we allow for more intelligent agents possessing a richer strategy set, and
we no longer constrain the estimate for the heterogeneity of the agents to a
probability space. We also introduce a scheme which allows the incorporation of
models with a wide variety of agent types, and discuss a mechanism for the
removal of bias from relevant parameters.

Information-Based Asset Pricing
  A new framework for asset price dynamics is introduced in which the concept
of noisy information about future cash flows is used to derive the price
processes. In this framework an as